In [2]:
import polars as pl
import pandas as pd
from datetime import datetime, timedelta, date 
semana = date.today().isocalendar().week

In [3]:
clubs = (pl.read_csv("https://raw.githubusercontent.com/IvoVillanueva/acb2026/refs/heads/main/logos_calendario/clubs2026.csv")
    .with_columns([
        pl.when(pl. col("equipo") == "Surne Bilbao Basket")
          .then(pl.lit("#9b51e0"))
          .otherwise(pl.col("color"))
          .alias("color")
          ]
    )
)
clubs

equipo,logo,color,url,abb
str,str,str,str,str
"""Barça""","""https://static.acb.com/img/www…","""#154284""","""https://www.acb.com/club/plant…","""BAR"""
"""Baskonia""","""https://static.acb.com/logos/1…","""#C8102E""","""https://www.acb.com/club/plant…","""BKN"""
"""Bàsquet Girona""","""https://static.acb.com/img/www…","""#B32836""","""https://www.acb.com/club/plant…","""GIR"""
"""BAXI Manresa""","""https://static.acb.com/img/31/…","""#D5001E""","""https://www.acb.com/club/plant…","""BAX"""
"""Casademont Zaragoza""","""https://static.acb.com/img/www…","""#C8102E""","""https://www.acb.com/club/plant…","""CAZ"""
…,…,…,…,…
"""Río Breogán""","""https://static.acb.com/logos/1…","""#85ADEA""","""https://www.acb.com/club/plant…","""BRE"""
"""Surne Bilbao Basket""","""https://static.acb.com/img/www…","""#9b51e0""","""https://www.acb.com/club/plant…","""SBB"""
"""UCAM Murcia""","""https://static.acb.com/img/www…","""#A51A14""","""https://www.acb.com/club/plant…","""UCM"""


In [4]:
data_europecup = (
    pl.read_csv(
        "https://raw.githubusercontent.com/IvoVillanueva/BOXSCORES-EUROLEAGE-2025_26/refs/heads/main/data/boxscores_fiba_europecup_2025-26.csv"
    )
    .with_columns(
        (
            pl.col("TP")
              .str.split(":")
              .list.get(0)
              .cast(pl.Int64)
            +
            pl.when(
                pl.col("TP")
                  .str.split(":")
                  .list.get(1)
                  .cast(pl.Int64) >= 30
            )
            .then(1)
            .otherwise(0)
        ).alias("minutes")
    )
    .filter(
     (pl.col("week") == semana)  
    )
    .select([
        "player",
        "equipo",
        pl.col("player").n_unique().alias("jug"),
        pl.col("minutes").alias("min"),
        pl.col("PTS").alias("pts"),
        pl.col("REB").alias("reb"),
        pl.col("AS").alias("ast"),
        pl.col("ST").alias("br"),
        pl.col("TO").alias("bp"),
        pl.col("BS").alias("tap"),
        pl.col("PF").alias("fa"),
        pl.col("EFF").alias("val"),
        pl.col("PM").alias("mm"),
    ])
    .join(clubs, on="equipo")
)
    

data_europecup


player,equipo,jug,min,pts,reb,ast,br,bp,tap,fa,val,mm,logo,color,url,abb
str,str,u32,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,str,str,str
